# <font color = 'indianred'>**Emotion Detection - MultiLabel** </font>

**Plan**

1. Set Environment
2. Load Dataset
3. Load Pre-trained Tokenizer
4. Train Model
     1. Compute Metric Function <br>
     2. Training Arguments <br>
     3. Specify Model
     4. Instantiate Trainer <br>
     5. Setup WandB <br>
     6. Training and Validation






















# <font color = 'indianred'> **1. Setting up the Environment** </font>



In [1]:
# If in Colab, then import the drive module from google.colab
import sys
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  # Mount the Google Drive to access files stored there
  drive.mount('/content/drive')

  # Install the latest version of torchtext library quietly without showing output
  # !pip install torchtext -qq
  !pip install transformers evaluate wandb datasets accelerate peft bitsandbytes setfit -U -qq ## NEW LINES ##
  basepath = '/content/drive/MyDrive/Colab_Notebooks/BUAN_6342_Applied_Natural_Language_Processing'
  sys.path.append('/content/drive/MyDrive/Colab_Notebooks/BUAN_6342_Applied_Natural_Language_Processing/0_Custom_files')
else:
  basepath = '/Users/harikrishnadev/Library/CloudStorage/GoogleDrive-harikrish0607@gmail.com/My Drive/Colab_Notebooks/BUAN_6342_Applied_Natural_Language_Processing/'
  sys.path.append('/Users/harikrishnadev/Library/CloudStorage/GoogleDrive-harikrish0607@gmail.com/My Drive/Colab_Notebooks/BUAN_6342_Applied_Natural_Language_Processing/0_Custom_files')

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.3/277.3 kB 29.2 MB/s 

<font color = 'indianred'> *Load Libraries* </font>

In [2]:
# standard data science librraies for data handling and v isualization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import multilabel_confusion_matrix, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import numpy as np

import torch
import torch.nn as nn
from datasets import load_dataset
from transformers import (
    TrainingArguments,
    Trainer,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoConfig,
    BitsAndBytesConfig,
)

import wandb
import evaluate

In [3]:
from peft import (
    TaskType,
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
)

In [4]:
# from sentence_transformers.losses import CosineSimilarityLoss

# from setfit import SetFitModel, SetFitTrainer

# <font color = 'indianred'> **2. Load_Dataset** </font>

In [4]:
emotion_data = load_dataset('harpreetmann/train_emotion_spring_2024')

Generating train split:   0%|          | 0/6179 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/1545 [00:00<?, ? examples/s]

In [5]:
emotion_data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 6179
    })
    valid: Dataset({
        features: ['text', 'label'],
        num_rows: 1545
    })
})

In [7]:
emotion_data['train'][0:2]

{'text': ['Does anyone know, are both Sims in a dual sim phone both locked to the same network! #worry',
  'A not very young man compiled info on a smiling baby then a mattress created new evil.'],
 'label': [[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0],
  [1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]}

In [6]:
labels = ['anger',
 'anticipation',
 'disgust',
 'fear',
 'joy',
 'love',
 'optimism',
 'pessimism',
 'sadness',
 'surprise',
 'trust']

In [18]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="michellejieli/emotion_text_classifier")
classifier_labels = labels

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [22]:
emotion_data['train'][:5]

{'text': ['Does anyone know, are both Sims in a dual sim phone both locked to the same network! #worry',
  'A not very young man compiled info on a smiling baby then a mattress created new evil.',
  '@TheSGJournal Step 1: Get rid of @LittleStokePark Step 2: Prioritise football Step 3: profit? #depressing',
  "You know when you're just slightly high, the state between sober and high af? That's beautiful. I wish I could feel like that 24/7.",
  '@OilersFanSean yup I feel dirty cheering'],
 'label': [[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0],
  [1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
  [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]]}

In [20]:
%time
predicted_labels = classifier(emotion_data['train']['text'][:5], classifier_labels)
predicted_labels

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.39 µs


[{'sequence': 'Does anyone know, are both Sims in a dual sim phone both locked to the same network! #worry',
  'labels': ['surprise',
   'anticipation',
   'love',
   'optimism',
   'fear',
   'joy',
   'trust',
   'disgust',
   'sadness',
   'pessimism',
   'anger'],
  'scores': [0.16518637537956238,
   0.1299232691526413,
   0.11385529488325119,
   0.1115543320775032,
   0.08780013024806976,
   0.07753915339708328,
   0.07642814517021179,
   0.07529307156801224,
   0.06401269882917404,
   0.054078105837106705,
   0.04432941973209381]},
 {'sequence': 'A not very young man compiled info on a smiling baby then a mattress created new evil.',
  'labels': ['surprise',
   'anger',
   'disgust',
   'anticipation',
   'optimism',
   'love',
   'fear',
   'trust',
   'pessimism',
   'sadness',
   'joy'],
  'scores': [0.8210710883140564,
   0.031187117099761963,
   0.026849981397390366,
   0.022637005895376205,
   0.017605604603886604,
   0.017213797196745872,
   0.015631282702088356,
   0.0137

In [9]:
predicted = []
for text in emotion_data['train']['text'][:5]:
  predicted.append(classifier(text, candidate_labels))

In [12]:
predicted

[{'sequence': 'Does anyone know, are both Sims in a dual sim phone both locked to the same network! #worry',
  'labels': ['pessimism',
   'fear',
   'anticipation',
   'surprise',
   'sadness',
   'anger',
   'love',
   'disgust',
   'trust',
   'optimism',
   'joy'],
  'scores': [0.4651341736316681,
   0.3931485116481781,
   0.09303836524486542,
   0.04039573296904564,
   0.0027960878796875477,
   0.0013479181798174977,
   0.001183513319119811,
   0.0011392472079023719,
   0.0007690825150348246,
   0.0006386956083588302,
   0.0004085774999111891]},
 {'sequence': 'A not very young man compiled info on a smiling baby then a mattress created new evil.',
  'labels': ['disgust',
   'anticipation',
   'surprise',
   'pessimism',
   'fear',
   'anger',
   'sadness',
   'optimism',
   'trust',
   'joy',
   'love'],
  'scores': [0.21995306015014648,
   0.20265617966651917,
   0.1818302720785141,
   0.12009801715612411,
   0.10243717581033707,
   0.06738840788602829,
   0.04807116463780403,
   